In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import datetime
# print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [4]:
features = ['card_id','purchase_date','subsector_id','merchant_id',
            'merchant_category_id','state_id','city_id','purchase_amount']
df_train = pd.read_csv('../train.csv')
df_test = pd.read_csv('../test.csv')
df_hist_trans = pd.read_csv('../../../ELO/historical_transactions.csv',usecols = features)
df_new_merchant_trans = pd.read_csv('../../../ELO/new_merchant_transactions.csv', usecols = features)

In [5]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    del df['purchase_date']

In [6]:
def get_new_columns(name,aggs):
    #for for 写法 nice
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [7]:
i = 0
import gc
import time
for df in [df_new_merchant_trans,df_hist_trans]:
    aggs = {}
    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
                'state_id','city_id']:
        aggs[col] = ['nunique']
    aggs['purchase_amount'] = ['sum','mean']
    if i == 0:
        prefix = 'new_hist'
    else:
        prefix = 'hist'
    new_columns = get_new_columns(prefix,aggs)
    i += 1
    df_hist_trans_group = df.groupby('card_id').agg(aggs)
    df_hist_trans_group.columns = new_columns
    df_hist_trans_group.reset_index(drop=False,inplace=True)
    df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
    df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
    del df_hist_trans_group
    gc.collect()
    time.sleep(5)

In [8]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime(2018,2,1) - df['first_active_month']).dt.days

In [9]:
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns = [c for c in df_train.columns if c not in exclude_features ]
target = df_train['target']

In [10]:
len(df_train.columns)

34

In [11]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_hist_month_nunique,new_hist_hour_nunique,new_hist_weekofyear_nunique,new_hist_dayofweek_nunique,...,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_mean,dayofweek,weekofyear,month,elapsed_time
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,2.0,8.0,7.0,7.0,...,94,41,3,7,-165.968739,-0.638341,3,22,6,245
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,2.0,5.0,4.0,4.0,...,142,57,3,9,-210.006336,-0.600018,6,52,1,396
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,1.0,1.0,1.0,1.0,...,13,8,2,5,-29.167391,-0.678311,0,31,8,549
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,2.0,5.0,5.0,4.0,...,50,25,5,7,-49.491364,-0.642745,4,35,9,153
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,2.0,14.0,8.0,7.0,...,66,26,6,6,-48.687656,-0.366073,2,44,11,92


In [12]:
df_train.fillna(0,inplace = True)
df_test.fillna(0,inplace = True)
df_test.iloc[1000]

first_active_month                       2017-10-01 00:00:00
card_id                                      C_ID_f60d9fee4f
feature_1                                                  3
feature_2                                                  1
feature_3                                                  1
new_hist_month_nunique                                     2
new_hist_hour_nunique                                      9
new_hist_weekofyear_nunique                                5
new_hist_dayofweek_nunique                                 5
new_hist_year_nunique                                      1
new_hist_subsector_id_nunique                              7
new_hist_merchant_id_nunique                              11
new_hist_merchant_category_id_nunique                     10
new_hist_state_id_nunique                                  3
new_hist_city_id_nunique                                   6
new_hist_purchase_amount_sum                        -7.32795
new_hist_purchase_amount

In [13]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(df_train[df_train_columns])
X_test_minmax = min_max_scaler.fit_transform(df_test[df_train_columns])

C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [14]:
train = pd.DataFrame(X_train_minmax,columns = df_train_columns)
train['card_id'] = df_train.card_id
train['target'] = df_train.target
test = pd.DataFrame(X_test_minmax,columns = df_train_columns)
test['card_id'] = df_test.card_id
train.head()

,feature_1,feature_2,feature_3,new_hist_month_nunique,new_hist_hour_nunique,new_hist_weekofyear_nunique,new_hist_dayofweek_nunique,new_hist_year_nunique,new_hist_subsector_id_nunique,new_hist_merchant_id_nunique,...,hist_state_id_nunique,hist_city_id_nunique,hist_purchase_amount_sum,hist_purchase_amount_mean,dayofweek,weekofyear,month,elapsed_time,card_id,target
0,1.00,0.5,1.0,1.0,0.347826,0.7,1.000000,0.5,0.416667,0.211009,...,0.105263,0.090909,0.000320,2.136813e-07,0.500000,0.403846,0.454545,0.107268,C_ID_92a2005557,-0.820283
1,0.75,0.0,0.0,1.0,0.217391,0.4,0.571429,0.5,0.166667,0.055046,...,0.105263,0.121212,0.000312,2.901925e-07,1.000000,0.980769,0.000000,0.173380,C_ID_3d0044924f,0.392913
2,0.25,0.5,0.0,0.5,0.043478,0.1,0.142857,0.5,0.041667,0.009174,...,0.052632,0.060606,0.000342,1.338821e-07,0.000000,0.576923,0.636364,0.240368,C_ID_d639edf6cd,0.688056
3,0.75,1.0,0.0,1.0,0.217391,0.5,0.571429,0.5,0.208333,0.064220,...,0.210526,0.090909,0.000339,2.048895e-07,0.666667,0.653846,0.727273,0.066988,C_ID_186d6a6901,0.142495
4,0.00,1.0,0.0,1.0,0.608696,0.8,1.000000,0.5,0.416667,0.330275,...,0.263158,0.075758,0.000339,7.572579e-07,0.333333,0.826923,0.909091,0.040280,C_ID_cdbd2c0db2,-0.159749


In [15]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=500,n_jobs = 4)
knn.fit(train[df_train_columns]) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=4, n_neighbors=500, p=2, radius=1.0)

In [16]:
train_neigh = knn.kneighbors(train[df_train_columns], return_distance=False) 
test_neigh = knn.kneighbors(test[df_train_columns], return_distance=False) 
test_neigh[1000]

array([ 97313, 153484, 103696, 195223, 117086, 126834,  55285, 129722,
        15024,  88110,  56097,  85188, 190539, 199787, 181474, 130130,
       148459,  51199,  29173,  93502,  95786, 129017, 123607,  61518,
       104734,  62061, 150547,  64760, 145015, 117899,   3063, 119442,
        45546, 151393, 124775,  89600,  51753,  80411, 136163, 187323,
        48063, 178762, 136288, 183012, 142023, 161295, 164705,  99484,
        64684,  65568, 187745, 180431,  10015,  24887,  59155,  25868,
        76376, 112904,  96446,  42782,  82513,  41836, 120395,  13365,
       177293,  65426, 125562,  82822, 170916, 100456, 176127, 167907,
       181669, 188877,  65844,  14399,  74246, 146239,  70222, 146256,
       191222,  55769, 193285,  25807,   5116, 144606,  43054, 199130,
        27274, 153659,  29374,  29235,  68510,   4763, 199317, 143921,
       101169,  19067, 155498,  87846,  10155,  91929, 164081,  77584,
        18083, 178713, 111755, 122539, 196728,  62991, 133283,  89383,
      

In [29]:
test_neigh.shape

(123623, 500)

In [17]:
train.iloc[0].target

-0.8202826

In [18]:
train_target = np.zeros(len(df_train))

In [19]:
for i in range(len(df_train)):
    cur = train_neigh[i]
    target_mean = 0
    for index in cur:
        target_mean += train.iloc[index].target
    train_target[i] = target_mean / len(cur)
    if i % 1000 == 0:
        print(i)
df_train_target = pd.DataFrame(train_target)
df_train_target['card_id'] = df_train.card_id

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [20]:
test_target = np.zeros(len(df_test))
for i in range(len(df_test)):
    cur = test_neigh[i]
    target_mean = 0
    for index in cur:
        target_mean += train.iloc[index].target
    test_target[i] = target_mean / len(cur)
    if i % 1000 == 0:
        print(i)
df_test_target = pd.DataFrame(test_target)
df_test_target['card_id'] = df_test.card_id

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000


In [24]:
train_target.shape,test_target.shape

((201917,), (123623,))

In [27]:
# df_train_50_neastneighbor.columns = ['50nearstneighbor','card_id']
# df_test_50_neastneighbor.columns = ['50nearstneighbor','card_id']
df_train_target.to_pickle('train_target_500nearst_neighbor.pickle')
df_test_target.to_pickle('test_target_500nearst_neighbor.pickle')